Import the libraries needed

In [2]:
import pandas as pd
#from pandas.compat import StringIO
from pandas.tseries.offsets import DateOffset

from datetime import datetime
import psutil


import numpy as np
import os
import pyodbc
pyodbc.drivers()

import csv

from sklearn.preprocessing import StandardScaler
import seaborn as sns

pd.options.display.max_rows = 1000

# For the Yahoo Finance api
from pandas_datareader import data as pdr
import yfinance as yf
yf.pdr_override() 


C:\Users\m033\Anaconda3\lib\site-packages\requests\__init__.py:109: RequestsDependencyWarning: urllib3 (1.26.16) or chardet (3.0.0)/charset_normalizer (2.0.4) doesn't match a supported version!
  warnings.warn(


### Database connections and SQL to retrieve the various data needed.

In [3]:
SQL_server = 'xxx\yyy'
Reporting_database = 'zzz'
Reference_Data_database = 'aaa'

pd.set_option('max_row', None)
#pd.options.display.max_rows = 1000

# Set a value for the home folder.
home_folder = "."

# Set values for the various paths.
input_path = home_folder + "\data"

print('==========================')
now = datetime.now()
print(now)
print('==========================')
print(psutil.virtual_memory())
print('==========================')

2024-08-14 17:48:48.573105
svmem(total=16756752384, available=4236488704, percent=74.7, used=12520263680, free=4236488704)


# 1. Get supplementary policy features - Tax, Fees and Risk.

  -  Identified in Lit Review.

In [4]:
conn = pyodbc.connect(
         'DRIVER={ODBC Driver 17 for SQL Server};'
         'SERVER='+SQL_server+';'
         'DATABASE='+Reporting_database+';'
         'Trusted_Connection=yes;')

tax_query = f"""SELECT pol.Policy_Number         AS ContractNumber
                       ,CASE WHEN pol.Is_Lump_Sum_Taxation IS NULL THEN 0
                             ELSE pol.Is_Lump_Sum_Taxation 
                        END AS Lump_Sum_Taxation
                       ,pol.Tax_Notification_Type
                       ,pol.Insurance_Tax_Type
                       ,pol.Main_Tax_Country
                       ,pol.Secondary_Tax_Country
                       ,pol.Taxed_Retirement_Income
                       ,pol.Policy_Fee_Type_Desc
                       ,pol.Future_Risk
                 FROM Policy pol
                 INNER JOIN Product prd ON prd.Product_Id = pol.Product_Id
                 WHERE prd.Product_Code IN ('FKP','GNPP',GNRP')
                 ORDER BY pol.Policy_Number;"""

df_tax = pd.read_sql(tax_query, conn)

# Types and row count
# print stats.
print(len(df_tax.index))
now = datetime.now()
print(now)
print('==========================')

print(df_tax.dtypes)

# Turn off dislays to protect PII
#df_tax.head(5)

137245
2024-08-14 17:49:06.244474
ContractNumber              object
Lump_Sum_Taxation            int64
Tax_Notification_Type       object
Insurance_Tax_Type          object
Main_Tax_Country            object
Secondary_Tax_Country       object
Taxed_Retirement_Income    float64
Policy_Fee_Type_Desc        object
Future_Risk                 object
dtype: object


## Drop obsolette or repeated features.

## Check for nulls

In [5]:
df_tax.isna().any()

ContractNumber             False
Lump_Sum_Taxation          False
Tax_Notification_Type       True
Insurance_Tax_Type          True
Main_Tax_Country           False
Secondary_Tax_Country      False
Taxed_Retirement_Income     True
Policy_Fee_Type_Desc       False
Future_Risk                False
dtype: bool

## Replace any nulls before checking the distributions of values for each column.

In [6]:
df_tax['Tax_Notification_Type'] = df_tax['Tax_Notification_Type'].astype(str)
df_tax['Insurance_Tax_Type'] = df_tax['Insurance_Tax_Type'].fillna(0).apply(np.int64)
df_tax['Insurance_Tax_Type'] = df_tax['Insurance_Tax_Type'].fillna(0).apply(np.int64)



## Check the distributions of values for each column.

In [7]:
#df_Horizon_All_Features['Lump_Sum_Taxation'] = df_Horizon_All_Features['Lump_Sum_Taxation'].fillna(0).apply(np.int64)
print('Lump_Sum_Taxation       : ',df_tax['Lump_Sum_Taxation'].value_counts())
print('Tax_Notification_Type   : ',df_tax['Tax_Notification_Type'].value_counts())
print('Insurance_Tax_Type      : ',df_tax['Insurance_Tax_Type'].value_counts())
print('Main_Tax_Country        : ',df_tax['Main_Tax_Country'].value_counts())
print('Secondary_Tax_Country   : ',df_tax['Secondary_Tax_Country'].value_counts())
print('Taxed_Retirement_Income : ',df_tax['Taxed_Retirement_Income'].value_counts())
print('Policy_Fee_Type_Desc    : ',df_tax['Policy_Fee_Type_Desc'].value_counts())
print('Future_Risk             : ',df_tax['Future_Risk'].value_counts())




Lump_Sum_Taxation       :  0    137245
Name: Lump_Sum_Taxation, dtype: int64
Tax_Notification_Type   :  None    137245
Name: Tax_Notification_Type, dtype: int64
Insurance_Tax_Type      :  0    137245
Name: Insurance_Tax_Type, dtype: int64
Main_Tax_Country        :  Deutschland     137245
Name: Main_Tax_Country, dtype: int64
Secondary_Tax_Country   :  Deutschland     137245
Name: Secondary_Tax_Country, dtype: int64
Taxed_Retirement_Income :  0.0    118116
Name: Taxed_Retirement_Income, dtype: int64
Policy_Fee_Type_Desc    :  Ein               109572
Aus                14947
Nicht Relevant     10226
Reduziert           2500
Name: Policy_Fee_Type_Desc, dtype: int64
Future_Risk             :  kein    137245
Name: Future_Risk, dtype: int64


## Policy_Fee_Type_Desc is the only feature showing a distribution across multiple values and is the only feature with potential predictive attributes.

## Drop all non predictive features and write to Excel

In [8]:
df_tax = df_tax.drop(['Lump_Sum_Taxation','Tax_Notification_Type','Insurance_Tax_Type','Main_Tax_Country','Secondary_Tax_Country','Taxed_Retirement_Income','Future_Risk'],axis=1)

# Write out to a comma separated values file.

# giving directory name
filename = input_path + '\Tax_Fee_Risk_data.csv'
df_tax.to_csv(filename, encoding='utf-8', index=False)        

# Turn off dislays to protect PII
#df_tax.head(10)

In [9]:
%who_ls DataFrame 

['df_tax']

In [10]:
del df_tax



In [11]:
print('==========================')
now = datetime.now()
print(now)
print('==========================')
print(psutil.virtual_memory())
print('==========================')

2024-08-14 17:49:10.829355
svmem(total=16756752384, available=4549464064, percent=72.8, used=12207288320, free=4549464064)
